In [1]:

!pip install bs4

You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.8/bin/python3 -m pip install --upgrade pip' command.


### Importing Packages

In [2]:

from bs4 import BeautifulSoup
import requests as rq
import pandas as pd

### Extracting data from wikipedia

In [3]:
tr_url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
data= rq.get(tr_url)
html_txt=data.text

### Webscraping the text to extract table contents from the html usig BeautifulSoup 

In [4]:
soup = BeautifulSoup(html_txt,"html5lib")

table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['Postal Code'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [5]:
df.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [6]:
df.shape

(103, 3)

### Load Geospatial coordinates data into a data frame

In [7]:
geo_cord=pd.read_csv('Geospatial_Coordinates.csv')
geo_cord.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Merge the data with existing postal code data 

In [8]:
geo_data=df.merge(geo_cord, on='Postal Code')
geo_data.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


In [9]:
geo_data.shape

(103, 5)

In [10]:
conda install -c conda-forge folium

Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [11]:
import folium 

In [12]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [14]:
from geopy.geocoders import Nominatim

### Get Latitude and Longitude of Toronto

In [17]:

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode('Toronto')
lat = location.latitude
lon = location.longitude
print('The coordinate of Toronto are {}, {}.'.format(lat, lon))

The coordinate of Toronto are 43.6534817, -79.3839347.


### Create Map and add markers

In [24]:

to_map = folium.Map(location=[lat, lon], zoom_start=10)

for lat, lng, borough, neighborhood in zip(geo_data['Latitude'], geo_data['Longitude'], geo_data['Borough'], geo_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#ff0000',
        fill_opacity=0.7,
        parse_html=False).add_to(to_map)  
    
to_map